# Your First 10 Minutes with llama.cpp

**Welcome!** You're about to run your first large language model locally on your machine. This is exciting!

**Time to complete:** 10 minutes  
**Prerequisites:** Basic command line familiarity  
**What you'll learn:**
- How to build llama.cpp
- Download your first model
- Generate text with a language model
- Troubleshoot common issues

**Confidence boost:** By the end of this tutorial, you'll have a working local LLM generating text. Let's go!

## Step 1: Verify Your Environment

First, let's make sure we're in the right place and have the necessary tools.

In [ ]:
import os
import subprocess
import sys
from pathlib import Path

# Check if we're in the llama.cpp directory
llama_cpp_root = Path.cwd()
while llama_cpp_root.name != "llama.cpp-learn" and llama_cpp_root != llama_cpp_root.parent:
    llama_cpp_root = llama_cpp_root.parent

if llama_cpp_root.name != "llama.cpp-learn":
    print("⚠️  Please run this notebook from within the llama.cpp-learn directory")
    llama_cpp_root = Path("/home/user/llama.cpp-learn")

print(f"✅ Working directory: {llama_cpp_root}")
print(f"✅ Python version: {sys.version}")

# Check for required tools
try:
    result = subprocess.run(['cmake', '--version'], capture_output=True, text=True)
    print(f"✅ CMake found: {result.stdout.split()[2]}")
except FileNotFoundError:
    print("❌ CMake not found. Please install CMake to continue.")

try:
    result = subprocess.run(['make', '--version'], capture_output=True, text=True)
    print(f"✅ Make found")
except FileNotFoundError:
    print("❌ Make not found. Please install build tools to continue.")

## Step 2: Build llama.cpp

Now let's compile llama.cpp. This creates the executable programs we'll use to run models.

**What's happening?** CMake configures the build for your system, and make compiles the C++ code into executables.

**Time estimate:** 2-3 minutes

In [ ]:
os.chdir(llama_cpp_root)

# Create build directory
build_dir = llama_cpp_root / "build"
build_dir.mkdir(exist_ok=True)

print("🔨 Configuring build with CMake...")
result = subprocess.run(
    ['cmake', '..', '-DCMAKE_BUILD_TYPE=Release'],
    cwd=build_dir,
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print("✅ CMake configuration successful")
else:
    print("❌ CMake configuration failed:")
    print(result.stderr)
    raise Exception("Build configuration failed")

print("\n🔨 Compiling llama.cpp... (this may take 2-3 minutes)")
result = subprocess.run(
    ['cmake', '--build', '.', '--config', 'Release', '--parallel'],
    cwd=build_dir,
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print("✅ Build successful!")
    
    # Check for the main executable
    main_executable = build_dir / "bin" / "llama-cli"
    if not main_executable.exists():
        main_executable = build_dir / "bin" / "main"  # Older versions
    
    if main_executable.exists():
        print(f"✅ Main executable found: {main_executable}")
    else:
        print("⚠️  Executable not in expected location. Checking build/bin/...")
        for exe in (build_dir / "bin").glob("*"):
            print(f"   Found: {exe.name}")
else:
    print("❌ Build failed:")
    print(result.stderr[-1000:])  # Last 1000 chars of error
    raise Exception("Build failed")

## Step 3: Download Your First Model

Let's download a small, efficient model to get started. We'll use **TinyLlama-1.1B** - it's small (around 600MB) but surprisingly capable!

**Why TinyLlama?**
- Small size: Quick to download
- Fast inference: Runs on any hardware
- Good quality: Surprisingly coherent for its size
- Perfect for learning!

**File format:** We're downloading the GGUF format (Q4_K_M quantization), which balances quality and size.

In [ ]:
import urllib.request
from tqdm.notebook import tqdm

# Create models directory
models_dir = llama_cpp_root / "models"
models_dir.mkdir(exist_ok=True)

# Model details
model_name = "tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf"
model_path = models_dir / model_name
model_url = f"https://huggingface.co/TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF/resolve/main/{model_name}"

if model_path.exists():
    print(f"✅ Model already downloaded: {model_path}")
    print(f"   Size: {model_path.stat().st_size / 1024 / 1024:.1f} MB")
else:
    print(f"📥 Downloading {model_name}...")
    print(f"   From: {model_url}")
    print(f"   To: {model_path}")
    print("   This will take 2-5 minutes depending on your connection...")
    
    class DownloadProgressBar(tqdm):
        def update_to(self, b=1, bsize=1, tsize=None):
            if tsize is not None:
                self.total = tsize
            self.update(b * bsize - self.n)
    
    try:
        with DownloadProgressBar(unit='B', unit_scale=True, miniters=1, desc=model_name) as t:
            urllib.request.urlretrieve(model_url, model_path, reporthook=t.update_to)
        print(f"\n✅ Download complete! Size: {model_path.stat().st_size / 1024 / 1024:.1f} MB")
    except Exception as e:
        print(f"❌ Download failed: {e}")
        print("\n💡 Troubleshooting:")
        print("   - Check your internet connection")
        print("   - Try downloading manually from HuggingFace")
        print(f"   - URL: {model_url}")
        raise

## Step 4: Your First Text Generation!

The moment you've been waiting for - let's generate some text!

We'll use the `llama-cli` program with some basic parameters:
- `-m`: Model file path
- `-p`: Prompt (what you want the model to write about)
- `-n`: Number of tokens to generate
- `--temp`: Temperature (creativity level, 0.7 is balanced)

In [ ]:
# Find the llama-cli executable
llama_cli = build_dir / "bin" / "llama-cli"
if not llama_cli.exists():
    llama_cli = build_dir / "bin" / "main"  # Older versions

if not llama_cli.exists():
    print("❌ Could not find llama-cli executable")
    print(f"   Searched in: {build_dir / 'bin'}")
    print("   Available files:")
    for f in (build_dir / "bin").glob("*"):
        print(f"     - {f.name}")
else:
    print(f"✅ Found executable: {llama_cli}")
    
    # Simple prompt
    prompt = "Once upon a time, in a land of machine learning,"
    
    print(f"\n🤖 Generating text...\n")
    print(f"Prompt: {prompt}")
    print("=" * 70)
    
    # Run inference
    result = subprocess.run(
        [
            str(llama_cli),
            '-m', str(model_path),
            '-p', prompt,
            '-n', '100',  # Generate 100 tokens
            '--temp', '0.7',
            '-ngl', '0'  # No GPU layers for compatibility
        ],
        capture_output=True,
        text=True,
        cwd=llama_cpp_root
    )
    
    if result.returncode == 0:
        print(result.stdout)
        print("=" * 70)
        print("\n🎉 SUCCESS! You just ran your first local LLM!")
    else:
        print("❌ Generation failed:")
        print(result.stderr)
        print("\nSee troubleshooting section below.")

## Step 5: Try It Yourself!

Now it's your turn! Modify the prompt below and run your own generation.

In [ ]:
# YOUR TURN: Change this prompt to anything you like!
your_prompt = "Explain what a large language model is in simple terms:"

print(f"🤖 Your prompt: {your_prompt}\n")
print("=" * 70)

result = subprocess.run(
    [
        str(llama_cli),
        '-m', str(model_path),
        '-p', your_prompt,
        '-n', '150',
        '--temp', '0.7',
        '-ngl', '0'
    ],
    capture_output=True,
    text=True,
    cwd=llama_cpp_root
)

print(result.stdout)
print("=" * 70)

## Understanding What Just Happened

Congratulations! Let's break down what you just accomplished:

### 1. **You built llama.cpp from source**
   - Compiled C++ code optimized for your CPU
   - Created executables for running LLMs efficiently

### 2. **You downloaded a GGUF model**
   - GGUF = GPT-Generated Unified Format
   - Q4_K_M = 4-bit quantization (compressed for efficiency)
   - Went from ~4GB (original) to ~600MB (quantized)

### 3. **You ran inference**
   - Loaded the model into memory
   - Fed it a prompt
   - Generated new text token by token
   - All running locally on your machine!

### 4. **Key concepts you used:**
   - **Prompt:** Your input text
   - **Tokens:** Pieces of words the model processes
   - **Temperature:** Controls randomness (0 = deterministic, 1 = creative)
   - **Max tokens:** How much text to generate

## Common Issues & Solutions

### Problem: Build fails
**Solution:**
- Make sure you have CMake 3.14+ and a C++ compiler
- On Ubuntu/Debian: `sudo apt install build-essential cmake`
- On macOS: `brew install cmake`
- On Windows: Install Visual Studio with C++ tools

### Problem: Download fails
**Solution:**
- Check internet connection
- Download manually from [HuggingFace](https://huggingface.co/TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF)
- Place in `models/` directory

### Problem: Generation is slow
**Solution:**
- This is normal on CPU! TinyLlama should do 5-20 tokens/second
- For faster inference, see Tutorial 3 on GPU acceleration
- Smaller context = faster generation

### Problem: Nonsensical output
**Solution:**
- Try adjusting temperature (0.5-0.9 is usually good)
- Make prompts more specific
- TinyLlama is small - try a larger model for better quality

### Problem: Out of memory
**Solution:**
- TinyLlama needs ~1GB RAM
- Close other applications
- Try an even smaller model (Q2_K quantization)
- Reduce context size with `-c` parameter

## Quick Reference: Command Parameters

Here are the most useful parameters for `llama-cli`:

| Parameter | Description | Example |
|-----------|-------------|----------|
| `-m` | Model file path | `-m models/model.gguf` |
| `-p` | Prompt text | `-p "Hello world"` |
| `-n` | Max tokens to generate | `-n 100` |
| `--temp` | Temperature (0-2) | `--temp 0.7` |
| `-c` | Context size | `-c 2048` |
| `-ngl` | GPU layers | `-ngl 32` |
| `--top-p` | Nucleus sampling | `--top-p 0.9` |
| `--top-k` | Top-k sampling | `--top-k 40` |
| `--repeat-penalty` | Repetition penalty | `--repeat-penalty 1.1` |

Run `./build/bin/llama-cli --help` for the complete list!

## Experiment Time! 

Now that you have the basics, try these experiments:

### Experiment 1: Temperature
Generate the same prompt with different temperatures and compare:
- Temperature 0.1 (very focused)
- Temperature 0.7 (balanced)
- Temperature 1.5 (very creative)

In [ ]:
prompt = "The three laws of robotics are:"
temperatures = [0.1, 0.7, 1.5]

for temp in temperatures:
    print(f"\n{'='*70}")
    print(f"Temperature: {temp}")
    print(f"{'='*70}")
    
    result = subprocess.run(
        [str(llama_cli), '-m', str(model_path), '-p', prompt, 
         '-n', '80', '--temp', str(temp), '-ngl', '0'],
        capture_output=True, text=True
    )
    
    # Extract just the generated text (skip metadata)
    output_lines = result.stdout.split('\n')
    for line in output_lines:
        if not line.startswith('llama') and not line.startswith('log_') and line.strip():
            print(line)

### Experiment 2: Length
Try generating different lengths (tokens) and observe how the model maintains coherence.

In [ ]:
prompt = "Here's a recipe for chocolate chip cookies:"
lengths = [50, 150, 300]

for length in lengths:
    print(f"\n{'='*70}")
    print(f"Generating {length} tokens")
    print(f"{'='*70}")
    
    result = subprocess.run(
        [str(llama_cli), '-m', str(model_path), '-p', prompt,
         '-n', str(length), '--temp', '0.7', '-ngl', '0'],
        capture_output=True, text=True
    )
    
    print(result.stdout[:500])  # Show first 500 chars
    print("..." if len(result.stdout) > 500 else "")

## What You've Learned

In just 10 minutes, you've:

✅ Built llama.cpp from source  
✅ Downloaded and understood GGUF format  
✅ Generated your first text with a local LLM  
✅ Experimented with parameters  
✅ Learned troubleshooting basics  

**You now have:**
- A working llama.cpp installation
- Your first model ready to use
- Understanding of basic inference parameters
- Confidence to experiment further!

## What's Next?

### Continue Your Learning Journey:

1. **Tutorial 2: Understanding GGUF** (20-30 minutes)
   - Deep dive into model formats
   - Read model metadata programmatically
   - Compare different quantizations
   - Choose the right model for your needs

2. **Tutorial 3: Inference Parameters** (30-40 minutes)
   - Master temperature, top-p, top-k
   - Understand sampling strategies
   - Optimize for quality vs speed
   - Best practices for different tasks

### Try Different Models:
- **Llama 3.2 1B/3B** - Better quality, still fast
- **Phi-3** - Excellent reasoning for size
- **Qwen 2.5** - Multilingual capabilities

### Explore Advanced Topics:
- GPU acceleration (Module 3)
- Custom model fine-tuning (Module 5)
- Production deployment (Module 6)

### Join the Community:
- [llama.cpp GitHub](https://github.com/ggerganov/llama.cpp)
- [Discord](https://discord.gg/llama-cpp)
- [HuggingFace Models](https://huggingface.co/models?library=gguf)

---

## Congratulations!

You've taken your first steps into the world of local LLMs. The journey ahead is exciting - you can now:
- Run AI completely privately on your machine
- Experiment with cutting-edge models
- Build your own AI-powered applications
- Learn how these fascinating systems work

**Keep experimenting, stay curious, and have fun! ** 🚀